<h2>Wiki Game Bot</h2><br>
This notebook steps through a program designed to find an optimal solution for <a href='https://en.wikipedia.org/wiki/Wikipedia:Wiki_Game'>The Wiki Game</a>, where players compete to find the shortest path between 2 random wikipedia articles by clicking links within the page. 
Our strategy is to use <a href='https://nlp.stanford.edu/projects/glove/'>GloVe</a> word vectors to pick the links that match the target link the best. To measure similarity between links we are using the cosine similarity formula.
<br>
\begin{equation}
\cos ({\bf a},{\bf b})= {{\bf a} {\bf b} \over \|{\bf a}\| \|{\bf b}\|} = \frac{ \sum_{i=1}^{n}{{\bf a}_i{\bf b}_i} }{ \sqrt{\sum_{i=1}^{n}{({\bf a}_i)^2}} \sqrt{\sum_{i=1}^{n}{({\bf b}_i)^2}} }
\end{equation}
<br>
The perfect solution could be computed by checking every link on every page for the target link, however this is exponentially computationally expensive. To find a balance between computing speed and the depth of search we are using a proccess inspired by <a href='http://bradley.bradley.edu/~chris/searches.html#bs'>Beam Search</a>. By only searching the top n links (with the highest cosine similarity) the program is generally able to find a solution quickly. The program has certain limitations due to its use of the word embeddings. If the words of an article name aren't in the embedding dictionary then it's unable to converge on the target. If an article is a person, the embeddings are limited with names and may only converge on celebrities and famous names.

In [184]:
import numpy as np
import re 
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [185]:
def create_embeddings():
    global embeddings_index
    embeddings_index = {}

    #Open the 300 dimensional embedding as a readable file 
    with open('glove.6B.300d.txt', 'r', encoding='utf-8') as f:
        #Loop Through Each Line
        for line in f:
            #Lower & Strip the line, then split into a list
            glove_parsing = line.lower().strip().split()

            #Take the first element (which is a word)
            embedding_word = glove_parsing[0]

            #Then take the embedding weights and place them into an array
            embedding_weights = np.array(glove_parsing[1:])

            #Create a dictionary where keys correspond to a given word and the values are the given weights  
            embeddings_index[embedding_word] = embedding_weights
    return embeddings_index

In [186]:
def retrive_wiki_links(url):
    """Given a string input, url, returns a list of all blue article links"""
    
    #Retrive the Entire html of the Wiki Page and Convert to a String from Bytes  
    html = str(urlopen(url).read())

    #Find the Start of the Content Page on the html
    index_start = html.index('id="mw-content-text"')
    
    #Find Where the End of the Content Page is, this is Usually Where the Reference Header Begins
    #By Doing this we also Avoid Picking up the Links in the Reference Section 
    index_end = None
    if ('id="References"') in html:
        index_end = html.index('id="References"')
    #Sometimes a Note Secection Exists, Which Comes Before the Reference Section, which also Contains Links we Wish to Avoid
    if ('id="Notes"') in html:
        index_end = html.index('id="Notes"')

    #Concatenate the html to only include the Content Section 
    html_clean = html[:index_start+1] + html[:index_end]

    #Process the html data using the Beautiful Soup library 
    cleaned_data = BeautifulSoup(html_clean, 'html.parser')

    #Retrive a list of all anchor tags 
    tags = cleaned_data('a')

    #Retrive href links 
    tag_list = []
    for tag in tags:
        tag = tag.get('href')
        tag_list.append(tag)

    #Remvoe None type elements
    tag_list = [x for x in tag_list if x != None]

    #Loop through the tags and filter
    wiki_links = []
    for tag in tag_list:
        #Blue links have no unique identifiers compared to other links, therefore other link types must be removed
        removed_terms = ['.jpg', '.jpeg', '/User:', '/wiki/Talk', '.png', '/wiki/Wikipedia',
                         '/wiki/Help', '/wiki/File', '/wiki/Template', '/wiki/Special', 'https://',
                         'disambiguation']
        if ('/wiki/') in tag and not any(substring in tag for substring in removed_terms):
            wiki_links.append(tag)
    return wiki_links

In [187]:
def wiki_link_cleaner(link):
    """Clean the raw link into words readable by the encodings"""
    
    link = re.sub('^(.*?)\/wiki\/', '', link) 
    link = link.replace('_', ' ').split('#')[0]
    link = re.sub(r'[^\w\s]',' ', link)
    return link 

In [188]:
def words_to_embeddings(wiki_links):
    """given a list of links(wiki_links), return them as the embedding of the words"""
    
    #Strip the links down to the keywords
    clean_links = []
    for link in wiki_links:
        link = wiki_link_cleaner(link)
        clean_links.append(link)
    
    #Loop through the links to make embeddings
    embedding_tags = []
    for link in clean_links:
        words = link.split()

        #If the word isn't within the embeddings, it is removed
        for word in words:
            if embeddings_index.get(word.lower()) is None:
                words.remove(word)
    
        #Create a 2d array (length of words by the dimesnion of the word embedding) 
        word_vecs = np.zeros((len(words), 300))

        #Add the respective weights to the array
        for i in range(len(words)):
            word_vecs[i] =  embeddings_index.get(words[i].lower())

        #Take the mean of the weights (used for multi word links)
        #It's possible to use a more complex method of multi-word embeddings
        embedding_tags.append(np.mean(word_vecs, axis=0))

    embedding_tags = np.array(embedding_tags)
    return embedding_tags

In [189]:
#Function used to determine the similarity between words 
def cosine_similarity(a, b):
    """Calulates the cosine similarity between vectors a and b"""
    
    numerator = np.dot(a, b)
    denominator = np.dot(np.linalg.norm(a), np.linalg.norm(b))
    return numerator / denominator


In [190]:

def beam_search(target, embedding_tags, BEAM_VALUE):
    """Uses the principles of a beam search algorithm to find the best path to a range of new links"""
    
    tag_values = []
    target = target.lower()
    words = target.split()

    #If the word isn't within the embeddings, it is removed
    for word in words:
        if embeddings_index.get(word) is None:
            words.remove(word)
  
    #Create a 2d array (length of words by the dimesnion of the word embedding) 
    word_vecs = np.zeros((len(words), 300))

    #Add the respective weights to the array
    for i in range(len(words)):
        word_vecs[i] =  embeddings_index.get(words[i])

    #Take the mean of the weights (used for multi word links)
    target_embedding = np.mean(word_vecs, axis=0)

    #Calculate the cosine similarities
    for i in range(len(embedding_tags)):
        tag_values.append(cosine_similarity(embedding_tags[i], target_embedding))

    for j in range(len(tag_values)):
        if np.isnan(tag_values[j]):
            tag_values[j] = 0

    #Beam value adjusted if is larger than the number of values
    temp = 0
    if len(tag_values) < BEAM_VALUE:
        temp = BEAM_VALUE
        BEAM_VALUE = len(tag_values)

    #Output the top BEAM_VALUE number of links
    output = np.argpartition(np.array(tag_values), -BEAM_VALUE)[-BEAM_VALUE:]

    BEAM_VALUE = temp
    return output


In [191]:
def dirty_to_searchable(index, word_list, return_link = False):
    """takes a half link and makes it a full wikipedia link"""
    
    index = int(index)
    link = word_list[index]
    if return_link == True:
        link = 'https://en.wikipedia.org'+link
    return link

In [192]:
def searchable_to_dirty(link):
    """removes the wikipedia part of the link"""
    return link.split(".wikipedia.org")[-1]

In [193]:
def wiki_game(starting_link, target_link):
    target = wiki_link_cleaner(target_link)
    blacklist = []
    link_position = {}
    number_of_searches = 0
    searches = []
    searches_history = []
    beam = 8

    #initial page search
    wiki_links = retrive_wiki_links(starting_link)

    #Create embeddings of the links
    embedding_tags = words_to_embeddings(wiki_links)
    
    #Run beam search to find the best embeddings (best 12 links on the first page)
    beam_initial_positions = beam_search(target, embedding_tags, 12)

    #Format new links from the beam search
    for i in beam_initial_positions:
        searches.append(dirty_to_searchable(i, wiki_links, True))
    searches_history.append(searches)
    
    #Keep checking new links until the target_link is found
    print('This may take a minute...')
    while target_link not in searches:
        positions_beam_1 = []
        total_links_beam_1 = []
        total_links_beam_2 = []
        number_of_searches += 1 
        
        #Limit the number of searches to prevent an infinite loop
        if number_of_searches > 9:
            print('Max search limit exceeded')
            is_successful = False
            break
        
        #Filter out links that have already been used to prevent looping
        for i in searches:
            wiki_links = retrive_wiki_links(i)
            for wiki_link in wiki_links:
                while wiki_links.count(wiki_link) != 1:
                    wiki_links.remove(wiki_link)
                if wiki_link in blacklist:
                    wiki_links.remove(wiki_link)
                else:
                    total_links_beam_1.append(wiki_link)
                    blacklist.append(wiki_link)

        for x in total_links_beam_1:
            while total_links_beam_1.count(x) != 1:
                total_links_beam_1.remove(x)

        #Beam search starts by picking top 8 links, and increases its beam by 3 each iteration
        embedding_tags = words_to_embeddings(total_links_beam_1)
        positions_beam_1 = beam_search(target, embedding_tags, beam)
        beam += 3
        
        #format the new links and add them to searches_history for the next iteration
        searches = []
        for position in positions_beam_1:
            link = dirty_to_searchable(position, total_links_beam_1, True)
            searches.append(link) 
        searches_history.append(searches)
        print('searching...')
        
    return searches_history, starting_link, target_link

In [194]:

def track_link_path(searches_history, starting_link, target_link):
    """Works backwards from the solution, using searches_history to find a path to the starting_link"""
    
    path = []
    term = searchable_to_dirty(target_link)
    path.append(target_link)
    for j in range(len(searches_history))[:-1][::-1]:
        for i in searches_history[j]:
            wiki_links = retrive_wiki_links(i)
            for link in wiki_links:
                if link == term:
                    path.append(i)
                    term = searchable_to_dirty(i)
                    break
            else:
                continue
            break
    path.append(starting_link)
    return path[::-1]


In [195]:
embeddings_index = create_embeddings() #Takes a minute to run

In [182]:
starting_link = input('Starting Link --- ')
target_link = input('Target Link --- ')

searches_history, starting_link, target_link = wiki_game(starting_link, target_link)
path = track_link_path(searches_history, starting_link, target_link)

for i in range(len(path)-1):
    print(path[i], end = " ---> ")
print(path[-1])


Starting Link --- https://en.wikipedia.org/wiki/Apple_pie
Target Link --- https://en.wikipedia.org/wiki/Shrek
This may take a minute...
searching...
searching...
searching...
https://en.wikipedia.org/wiki/Apple_pie ---> https://en.wikipedia.org/wiki/Ginger ---> https://en.wikipedia.org/wiki/Gingerbread ---> https://en.wikipedia.org/wiki/Gingerbread_man ---> https://en.wikipedia.org/wiki/Shrek


In [178]:
starting_link = 'https://en.wikipedia.org/wiki/Interference_(communication)'
target_link = 'https://en.wikipedia.org/wiki/Popcorn'

searches_history, starting_link, target_link = wiki_game(starting_link, target_link)
path = track_link_path(searches_history, starting_link, target_link)

for i in range(len(path)-1):
    print(path[i], end = " ---> ")
print(path[-1])


This may take a minute...
searching...
searching...
searching...
https://en.wikipedia.org/wiki/Interference_(communication) ---> https://en.wikipedia.org/wiki/Wireless_networks ---> https://en.wikipedia.org/wiki/Wi-Fi ---> https://en.wikipedia.org/wiki/Microwave_oven ---> https://en.wikipedia.org/wiki/Popcorn


In [180]:
starting_link = 'https://en.wikipedia.org/wiki/Early_Christianity'
target_link = 'https://en.wikipedia.org/wiki/Eminem'

searches_history, starting_link, target_link = wiki_game(starting_link, target_link)
path = track_link_path(searches_history, starting_link, target_link)

for i in range(len(path)-1):
    print(path[i], end = " ---> ")
print(path[-1])

This may take a minute...
searching...
searching...
https://en.wikipedia.org/wiki/Early_Christianity ---> https://en.wikipedia.org/wiki/Christian_music ---> https://en.wikipedia.org/wiki/Music_download ---> https://en.wikipedia.org/wiki/Eminem


In [183]:
starting_link = 'https://en.wikipedia.org/wiki/Apple_pie'
target_link = 'https://en.wikipedia.org/wiki/Shrek'

searches_history, starting_link, target_link = wiki_game(starting_link, target_link)
path = track_link_path(searches_history, starting_link, target_link)

for i in range(len(path)-1):
    print(path[i], end = " ---> ")
print(path[-1])

This may take a minute...
searching...
searching...
searching...
https://en.wikipedia.org/wiki/Apple_pie ---> https://en.wikipedia.org/wiki/Ginger ---> https://en.wikipedia.org/wiki/Gingerbread ---> https://en.wikipedia.org/wiki/Gingerbread_man ---> https://en.wikipedia.org/wiki/Shrek
